In [ ]:
import os
import asyncio
from dotenv import load_dotenv
from typing import TYPE_CHECKING, Annotated, Any
from agent_framework import tool, Message, AgentResponse
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity.aio import AzureCliCredential
from random import randrange

if TYPE_CHECKING:
    from agent_framework import SupportsAgentRun

In [ ]:
@tool(approval_mode="always_require")
def get_weather(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get the current weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

In [ ]:
@tool(approval_mode="always_require")
def get_weather_detail(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get the current weather for a given location."""
    conditions = ["sunny", "cloudy", "raining", "snowing", "clear"]

    return (
        f"The weather in {location} is {conditions[randrange(0, len(conditions))]} and {randrange(-10, 30)}°C, "
        "with a humidity of 88%. "
        f"Tomorrow will be {conditions[randrange(0, len(conditions))]} with a high of {randrange(-10, 30)}°C."
    )


In [ ]:
load_dotenv(override=True)

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")

In [ ]:
credential = AzureCliCredential()
client = AzureOpenAIResponsesClient(
    project_endpoint=project_endpoint,
    deployment_name=model,
    credential=credential,
)

agent = client.as_agent(
    name="WeatherAgent",
    instructions="""
        You are a helpful weather assistant. 
        Use the get_weather tool to provide weather information.
        Use the get_weather_detail tool to provide detailed weather information.
        """,
    tools=[get_weather, get_weather_detail],
)

In [ ]:
async def handle_approvals(query: str, agent: "SupportsAgentRun") -> AgentResponse:
    result = await agent.run(query)
    
    while len(result.user_input_requests) > 0:
        # Start with the original query
        new_inputs: list[Any] = [query]

        for user_input_needed in result.user_input_requests:
            print(
                f"\nUser Input Request for function from {agent.name}:"
                f"\n  Function: {user_input_needed.function_call.name}"
                f"\n  Arguments: {user_input_needed.function_call.arguments}"
            )

            # Add the assistant message with the approval request
            new_inputs.append(Message("assistant", [user_input_needed]))

            # Get user approval
            user_approval = await asyncio.to_thread(input, "\nApprove function call? (y/n): ")

            # Add the user's approval response
            new_inputs.append(
                Message("user", [user_input_needed.to_function_approval_response(user_approval.lower() == "y")])
            )

        # Run again with all the context
        result = await agent.run(new_inputs)

    return result

In [ ]:
query = "Can you give me an update of the weather in LA and Portland and detailed weather for Seattle?"
print(f"User: {query}")

result = await handle_approvals(query, agent)

In [ ]:
print(result)